In [16]:
import pandas as pd

pd.options.display.max_colwidth = 500

In [17]:
s3_input = 's3://tick-genome/dna/2018-06-28/pre-assembly_v3/'
tick1_r1 = 'tick_1_S1_R1_post-trimming.fastq.gz'
tick1_r2 = tick1_r1.replace("R1", "R2")

tick2_r1 = 'tick_2_S2_R1_post-trimming.fastq.gz'
tick2_r2 = tick2_r1.replace("R1", "R2")

read1 = dict(tick1=tick1_r1, tick2=tick2_r1)
read2 = dict(tick1=tick1_r2, tick2=tick2_r2)

data = dict(read1=read1, read2=read2)

reads = pd.DataFrame(data)
# samples['id'] = samples.index
reads.index.name = 'id'
reads = reads.applymap(lambda x: s3_input + x)
reads

,read1,read2
id,,
tick1,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz
tick2,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz


Try multiple kmer sizes for assembly

In [18]:
import itertools

ksizes = 75, 81, 85
ticks = 'tick1', 'tick2'

data = list(itertools.product(ksizes, ticks))

parameters = pd.DataFrame(data, columns=['ksize', 'tick'])
# parameters = parameters.set_index()
parameters

,ksize,tick
0,75,tick1
1,75,tick2
2,81,tick1
3,81,tick2
4,85,tick1
5,85,tick2


In [19]:

samples = reads.merge(parameters, left_index=True, right_on='tick')
samples

,read1,read2,ksize,tick
0,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,75,tick1
2,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,81,tick1
4,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,85,tick1
1,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,75,tick2
3,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,81,tick2
5,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,85,tick2


In [20]:
samples['id'] = samples['tick'] + '_ksize' + samples['ksize'].astype(str)
samples

,read1,read2,ksize,tick,id
0,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,75,tick1,tick1_ksize75
2,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,81,tick1,tick1_ksize81
4,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,85,tick1,tick1_ksize85
1,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,75,tick2,tick2_ksize75
3,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,81,tick2,tick2_ksize81
5,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,85,tick2,tick2_ksize85


In [21]:
samples['output'] = 's3://tick-genome/dna/2018-06-28/' + 'assembly_' + samples['id']
samples

,read1,read2,ksize,tick,id,output
0,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,75,tick1,tick1_ksize75,s3://tick-genome/dna/2018-06-28/assembly_tick1_ksize75
2,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,81,tick1,tick1_ksize81,s3://tick-genome/dna/2018-06-28/assembly_tick1_ksize81
4,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,85,tick1,tick1_ksize85,s3://tick-genome/dna/2018-06-28/assembly_tick1_ksize85
1,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,75,tick2,tick2_ksize75,s3://tick-genome/dna/2018-06-28/assembly_tick2_ksize75
3,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,81,tick2,tick2_ksize81,s3://tick-genome/dna/2018-06-28/assembly_tick2_ksize81
5,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,85,tick2,tick2_ksize85,s3://tick-genome/dna/2018-06-28/assembly_tick2_ksize85


In [22]:
samples = samples.set_index('id')
samples['id'] = samples.index
samples

,read1,read2,ksize,tick,output,id
id,,,,,,
tick1_ksize75,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,75,tick1,s3://tick-genome/dna/2018-06-28/assembly_tick1_ksize75,tick1_ksize75
tick1_ksize81,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,81,tick1,s3://tick-genome/dna/2018-06-28/assembly_tick1_ksize81,tick1_ksize81
tick1_ksize85,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_1_S1_R2_post-trimming.fastq.gz,85,tick1,s3://tick-genome/dna/2018-06-28/assembly_tick1_ksize85,tick1_ksize85
tick2_ksize75,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,75,tick2,s3://tick-genome/dna/2018-06-28/assembly_tick2_ksize75,tick2_ksize75
tick2_ksize81,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,81,tick2,s3://tick-genome/dna/2018-06-28/assembly_tick2_ksize81,tick2_ksize81
tick2_ksize85,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R1_post-trimming.fastq.gz,s3://tick-genome/dna/2018-06-28/pre-assembly_v3/tick_2_S2_R2_post-trimming.fastq.gz,85,tick2,s3://tick-genome/dna/2018-06-28/assembly_tick2_ksize85,tick2_ksize85


In [23]:
folder = '../assemble/'
! mkdir $folder

import json

samples.to_csv(f'{folder}/samples.csv', index=True)

config = dict(program='../reflow/assemble.rf', runs_file='samples.csv')

with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)

mkdir: ../assemble/: File exists
